In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xml.etree.ElementTree as ET
import glob
import os
from pathlib import Path
import random
import shutil
from tqdm import tqdm

%matplotlib inline
random.seed(0)

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
train_imgs = os.listdir("Images/train")
len(train_imgs)

300

In [4]:
train, val = train_test_split(train_imgs, test_size=0.1, random_state=42)
print(len(train), len(val))

270 30


In [5]:
def xml_to_yolo_bbox(bbox, w, h):
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

In [10]:
cwd = os.getcwd()
def creat_dataset(data, dataset_type):
    images_path = Path(f"table/images/{dataset_type}")
    images_path.mkdir(parents=True, exist_ok=True)
    labels_path = Path(f"table/labels/{dataset_type}")
    labels_path.mkdir(parents=True, exist_ok=True)
    xml_path =  Path("Annotation/train")
    
    for img in tqdm(data):
        shutil.copy(os.path.join(cwd,"Images","train",img), images_path)
        
        filename = img.split(".")[0]
        xml_name = filename + ".xml"
        xml_file = os.path.join(xml_path, xml_name)
        result, classes = [], []
        tree = ET.parse(xml_file)
        root = tree.getroot()
        width = int(root.find("size").find("width").text)
        height = int(root.find("size").find("height").text)
        for obj in root.findall('object'):
            label = obj.find("name").text
            if label not in classes:
                classes.append(label)
            index = classes.index(label)
            pil_bbox = [int(x.text) for x in obj.find("bndbox")]
            yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
            bbox_string = " ".join([str(x) for x in yolo_bbox])
            result.append(f"{index} {bbox_string}")
        if result:
            with open(os.path.join(labels_path, f"{filename}.txt"), "w", encoding="utf-8") as f:
                f.write("\n".join(result))

In [11]:
creat_dataset(train, "train")
creat_dataset(val, "val")

100%|██████████████████████████████████████████████████████████████████████████████| 30/30 [00:01<00:00, 17.44it/s]
